In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle

from my_classes import DataGeneratorSkillBorders

2024-06-11 09:50:15.144230: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 09:50:15.183498: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 09:50:15.831956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
video_border_labels_path = '../data/df_video_border_labels.pkl'
video_folder = '../videos/'

video_names = [
    '20240201_atelier_001.mp4',
    '20240201_atelier_002.mp4',
    '20240201_atelier_003.mp4',
    '20240201_atelier_004.mp4',
    '20240201_atelier_005.mp4',
    '20240209_atelier_006.mp4',
    '20240209_atelier_007.mp4',
    '20240209_atelier_008.mp4',
]

train_videos = [ video_folder + trainvid for trainvid in video_names]

In [6]:
def pickle_load_or_create(path, cols):
    if os.path.exists(path):
        with open(path, 'rb') as file:
            return pickle.load(file)
    else:
        return pd.DataFrame(columns=cols)

In [7]:
df_labels = pickle_load_or_create(video_border_labels_path, [])
df_labels

,path,frame,border
0,20240209_atelier_006.mp4,745,0
1,20240209_atelier_007.mp4,103,0
2,20240201_atelier_002.mp4,91,0
3,20240201_atelier_004.mp4,806,2
4,20240201_atelier_003.mp4,622,2
...,...,...,...
727,20240201_atelier_004.mp4,2057,9
728,20240201_atelier_001.mp4,517,2
729,20240201_atelier_001.mp4,215,0
730,20240201_atelier_001.mp4,633,9


In [8]:
df_labels.loc[df_labels.border == 5, 'border'] = 3
df_labels.loc[df_labels.border == 9, 'border'] = 4

In [9]:
def get_random_frame(videos, grey=True, scale=1):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
          0 : ground
          1 : heels of ground
          2 : air
          3 : Fault
          4 : no skipper or not jumping
    """
    path = videos[np.random.randint(0, len(videos)-1)]
    
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # frame = cv2.cvtColor(frame, 7)
    cap.release()
    # cv2.destroyAllWindows()

    return path, frame_nr, frame

In [10]:
input_shape = get_random_frame(train_videos, grey=True, scale=0.4)[2].shape
input_shape

(768, 432, 3)

In [11]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,320 (220.00 KB)

 Trainable params: 56,320 (220.00 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
unique_labels = df_labels['border'].unique()
unique_labels

array([0, 2, 1, 4, 3])

In [14]:
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5))

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       147,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,877 (796.39 KB)

 Trainable params: 203,877 (796.39 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from my_classes import DataGeneratorSkillBorders
# Parameters
params = {'dim': (64,64),
          'batch_size': 16,
          'n_classes': len(unique_labels),
          'n_channels': 3,
          'shuffle': True,
          'train': True}

training_generator = DataGeneratorSkillBorders(df_labels, video_folder=video_folder, **params)
test_generator = DataGeneratorSkillBorders(df_labels, video_folder=video_folder, train=False)

                         path  frame  border
180  20240209_atelier_006.mp4    308       1
638  20240201_atelier_004.mp4    715       0
482  20240201_atelier_003.mp4   1189       0
598  20240209_atelier_006.mp4    619       1
223  20240201_atelier_001.mp4    436       2
..                        ...    ...     ...
602  20240201_atelier_001.mp4     50       4
565  20240209_atelier_007.mp4   1062       0
73   20240201_atelier_004.mp4    568       0
445  20240201_atelier_001.mp4    492       0
666  20240201_atelier_002.mp4    506       2

[586 rows x 3 columns]
                         path  frame  border
512  20240201_atelier_005.mp4    753       0
228  20240209_atelier_007.mp4    692       0
129  20240201_atelier_004.mp4    982       0
710  20240209_atelier_006.mp4    384       2
300  20240201_atelier_004.mp4   1165       2
..                        ...    ...     ...
122  20240201_atelier_005.mp4    527       1
522  20240201_atelier_002.mp4    101       0
545  20240201_atelier_002.mp4  

In [17]:
%%time
X, y = training_generator.__getitem__(3)

CPU times: user 12.7 s, sys: 259 ms, total: 13 s
Wall time: 2.9 s


In [18]:
X

array([[[[0.35294118, 0.37647059, 0.41176471],
         [0.34901961, 0.38431373, 0.40784314],
         [0.34901961, 0.38431373, 0.40784314],
         ...,
         [0.30980392, 0.34509804, 0.36470588],
         [0.32941176, 0.35686275, 0.38039216],
         [0.34117647, 0.36470588, 0.4       ]],

        [[0.36078431, 0.38823529, 0.41176471],
         [0.36078431, 0.39607843, 0.41960784],
         [0.36862745, 0.39607843, 0.41960784],
         ...,
         [0.32156863, 0.34117647, 0.36470588],
         [0.34509804, 0.35686275, 0.37647059],
         [0.34901961, 0.36862745, 0.39215686]],

        [[0.34117647, 0.37254902, 0.39607843],
         [0.36470588, 0.4       , 0.42352941],
         [0.37254902, 0.4       , 0.42352941],
         ...,
         [0.29803922, 0.31764706, 0.34117647],
         [0.31372549, 0.34117647, 0.34509804],
         [0.32941176, 0.35294118, 0.36862745]],

        ...,

        [[0.61960784, 0.61960784, 0.61960784],
         [0.63529412, 0.62352941, 0.62745098]

In [19]:
y

array([0, 0, 0, 0, 0, 0, 4, 2, 2, 0, 2, 0, 4, 1, 0, 4])

In [20]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=10, 
                    validation_data=test_generator)

Epoch 1/10


2024-06-11 09:51:32.414633: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8


 1/36 ━━━━━━━━━━━━━━━━━━━━ 17:40 30s/step - accuracy: 0.6875 - loss: 1.5233

2024-06-11 09:51:48.808622: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


36/36 ━━━━━━━━━━━━━━━━━━━━ 307s 8s/step - accuracy: 0.5133 - loss: 1.3275 - val_accuracy: 0.4896 - val_loss: 1.2166
Epoch 2/10


...

36/36 ━━━━━━━━━━━━━━━━━━━━ 325s 8s/step - accuracy: 0.8477 - loss: 0.4731 - val_accuracy: 0.8594 - val_loss: 0.3925
Epoch 10/10


2024-06-11 10:39:52.238012: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3 of 8
2024-06-11 10:40:10.735174: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 7 of 8


 1/36 ━━━━━━━━━━━━━━━━━━━━ 20:52 36s/step - accuracy: 0.9375 - loss: 0.2191

2024-06-11 10:40:15.313175: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


36/36 ━━━━━━━━━━━━━━━━━━━━ 323s 8s/step - accuracy: 0.8716 - loss: 0.3534 - val_accuracy: 0.8733 - val_loss: 0.3318


In [26]:
pd.DataFrame(history.history)

,accuracy,loss,val_accuracy,val_loss
0,0.489583,1.291249,0.489583,1.216628
1,0.467014,1.236270,0.520833,1.156343
2,0.546875,1.150366,0.592014,1.014438
3,0.664931,0.964694,0.744792,0.779860
4,0.717014,0.756260,0.793403,0.685382
5,0.796875,0.629358,0.821181,0.549196
6,0.812500,0.530113,0.845486,0.459185
7,0.826389,0.491764,0.875000,0.415029
8,0.850694,0.451510,0.859375,0.392537
9,0.878472,0.352735,0.873264,0.331836


In [30]:
with open('../data/models/last_cnn_model_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)

FileNotFoundError: [Errno 2] No such file or directory: '../data/models/last_cnn_model_history.pkl'

In [22]:
df_labels = pickle_load_or_create(video_border_labels_path, [])
df_labels

,path,frame,border
0,20240209_atelier_006.mp4,745,0
1,20240209_atelier_007.mp4,103,0
2,20240201_atelier_002.mp4,91,0
3,20240201_atelier_004.mp4,806,2
4,20240201_atelier_003.mp4,622,2
...,...,...,...
727,20240201_atelier_004.mp4,2057,9
728,20240201_atelier_001.mp4,517,2
729,20240201_atelier_001.mp4,215,0
730,20240201_atelier_001.mp4,633,9
